In [1]:
import os
import sys

# Get the current notebook's directory
current_dir = os.getcwd()

# Go up one level to project root
project_root = os.path.abspath(os.path.join(current_dir, '..'))
print(project_root)
print(sys.path)
# Add project root to path
sys.path.insert(0, project_root)

print(sys.path)

/home/jovyan
['/home/jovyan/evaluation', '/opt/conda/lib/python311.zip', '/opt/conda/lib/python3.11', '/opt/conda/lib/python3.11/lib-dynload', '', '/opt/conda/lib/python3.11/site-packages']
['/home/jovyan', '/home/jovyan/evaluation', '/opt/conda/lib/python311.zip', '/opt/conda/lib/python3.11', '/opt/conda/lib/python3.11/lib-dynload', '', '/opt/conda/lib/python3.11/site-packages']


In [2]:
pip install elasticsearch sentence_transformers nltk tqdm

Note: you may need to restart the kernel to use updated packages.


In [3]:
from elasticsearch import Elasticsearch
# Connect to Elasticsearch
es = Elasticsearch("http://elasticsearch:9200")

# Check connection
if es.ping():
    print("Connected to Elasticsearch")
else:
    print("Could not connect to Elasticsearch")

Connected to Elasticsearch


In [4]:
from sentence_transformers import SentenceTransformer
import random
import torch
import numpy as np
from nltk.tokenize import sent_tokenize
import nltk
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter compatibility
import json
import re
import unicodedata

/opt/conda/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [18]:
# Download required NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
def old_generate_enhanced_embedding(text, prefix=''):
    """Generate enhanced embeddings with better context preservation"""
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    
    # Enhanced prompt engineering for different vector types
    if prefix == 'question':
        enhanced_text = f"Is this a question? {text}"
    elif prefix == 'text':
        enhanced_text = f"Here’s some background context: {text}"
    elif prefix == 'combined':
        enhanced_text = f"Summary of question and context: {text}" 
    else:
        enhanced_text = text
        
    # Generate embedding with temperature
    with torch.no_grad():
        embedding = model.encode(
            enhanced_text,
            normalize_embeddings=True,
            convert_to_tensor=True
        )
    
    return embedding.cpu().numpy()

In [7]:

def old_create_index():
    """Create an index with improved settings, custom analyzers, and dense_vector mappings."""
    if es.indices.exists(index="qa_index_2"):
        es.indices.delete(index="qa_index_2")
    
    mappings = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "refresh_interval": "1s"
            },
            "analysis": {
                "analyzer": {
                    "rag_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "stop",
                            "porter_stem",
                            "synonym",
                            "word_delimiter_graph"
                        ]
                    }
                },
                "filter": {
                    "synonym": {
                        "type": "synonym",
                        "synonyms": [
                            "wellness, health, wellbeing",
                            "stress, anxiety, tension",
                            "help, assist, support",
                            "question, query, ask"
                        ]
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "text": {
                    "type": "text",
                    "analyzer": "rag_analyzer",
                    "search_analyzer": "rag_analyzer"
                },
                "response": {"type": "text"},
                "question_vector_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                },
                "text_vector_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                },
                "question_text_vector_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                },
                "vector_combined_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                }
            }
        }
    }
    
    es.indices.create(index="qa_index_2", body=mappings)
    print("Index created with enhanced analyzers and k-NN vector fields.")


In [8]:
def old_index_data(data):
    """Index data with improved preprocessing, embedding generation, and progress tracking for Jupyter."""
    total_patterns = sum(len(intent['patterns']) for intent in data['intents'])  # Calculate total patterns
    progress_bar = tqdm(total=total_patterns, desc="Indexing Patterns", unit="pattern")
    
    for idx, intent in enumerate(data['intents']):
        for pattern in intent['patterns']:
            # Generate variations of the pattern
            variations = [
                pattern,
                f"Q: {pattern}",
                f"Question: {pattern}",
                pattern.lower()
            ]
            
            # Split pattern into sentences for chunking
            chunks = sent_tokenize(pattern)
            
            for chunk_idx, chunk in enumerate(chunks):
                # Generate embeddings with different contexts
                question_vector = generate_enhanced_embedding(chunk, prefix='question')
                text_vector = generate_enhanced_embedding(chunk, prefix='text')
                question_text_vector = generate_enhanced_embedding(chunk, prefix='combined')
                
                # Generate embeddings for variations and average them
                variation_embeddings = [generate_enhanced_embedding(v) for v in variations]
                combined_vector = np.mean(variation_embeddings, axis=0)

                # Index each chunk with all vectors
                es.index(index="qa_index_2", id=f"{idx}_{chunk_idx}", body={
                    'text': chunk,
                    'question_vector_knn': question_vector.tolist(),
                    'text_vector_knn': text_vector.tolist(),
                    'question_text_vector_knn': question_text_vector.tolist(),
                    'vector_combined_knn': combined_vector.tolist(),
                    'response': random.choice(intent['responses'])
                })

                if idx < 5:  # Adjust as needed to check initial items only
                    print(f"Pattern: {pattern}")
                    print(f"Question Vector: {question_vector[:5]}")  # Print partial embedding for readability
                    print(f"Text Vector: {text_vector[:5]}")
                    print(f"Combined Vector: {combined_vector[:5]}")
                    print("-" * 40)            
            
            # Update progress after each pattern
            progress_bar.update(1)
    
    progress_bar.close()
    print("Data indexed successfully with enhanced embeddings and variations.")


In [9]:
def old_clean_text(text):
    # Remove bullet points and any common symbols that might disrupt embeddings
    text = re.sub(r'\n+', ' ', text)  # Replace new lines with space
    text = re.sub(r'•|\*|•|-|—|–', '', text)  # Remove common bullet characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Remove non-alphanumeric characters
    return text


In [10]:
# Load data from intents.json
with open('/home/jovyan/data/intents.json') as f:
    data = json.load(f)


In [11]:
def clean_text(text):
    """
    Enhanced text cleaning function with additional preprocessing steps
    """
    # Normalize unicode characters
    text = unicodedata.normalize('NFKC', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove bullet points and any common symbols
    text = re.sub(r'\n+', ' ', text)  # Replace new lines with space
    text = re.sub(r'•|\*|•|-|—|–', '', text)  # Remove common bullet characters
    
    # Remove URLs
    text = re.sub(r'http\S+|www.\S+', '', text)
    
    # Remove email addresses
    text = re.sub(r'\S+@\S+', '', text)
    
    # Remove special characters but keep basic punctuation
    text = re.sub(r'[^a-z0-9\s.,!?]', '', text)
    
    # Replace multiple spaces with single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text


In [12]:
def generate_pattern_variations(pattern):
    """
    Generate meaningful variations of the input pattern
    """
    variations = [
        pattern,
        f"Q: {pattern}",
        f"Question: {pattern}",
        pattern.lower(),
        # Add question-specific variations
        f"Can you tell me {pattern.lower()}?",
        f"I want to know {pattern.lower()}",
        f"Please explain {pattern.lower()}"
    ]
    return list(set(variations))  # Remove any duplicates

In [13]:
def generate_enhanced_embedding(text, prefix='', model=None):
    """
    Generate enhanced embeddings with better context preservation and caching
    """
    if model is None:
        model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    
    # Enhanced prompt engineering for different vector types
    if prefix == 'question':
        enhanced_text = f"Question context: {text}"
    elif prefix == 'text':
        enhanced_text = f"Background information: {text}"
    elif prefix == 'combined':
        enhanced_text = f"Complete context: {text}"
    else:
        enhanced_text = text
        
    # Generate embedding with temperature
    with torch.no_grad():
        embedding = model.encode(
            enhanced_text,
            normalize_embeddings=True,
            convert_to_tensor=True
        )
    
    return embedding.cpu().numpy()


In [14]:
def index_data(data, es, index_name="qa_index_2"):
    """
    Index data with improved preprocessing, embedding generation, and progress tracking
    """
    # Initialize the model once to avoid reloading
    model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
    
    # Calculate total patterns for progress bar
    total_patterns = sum(len(intent['patterns']) for intent in data['intents'])
    progress_bar = tqdm(total=total_patterns, desc="Indexing Patterns", unit="pattern")
    
    # Track statistics for logging
    stats = {
        'total_patterns': total_patterns,
        'processed_patterns': 0,
        'failed_patterns': 0
    }
    
    for idx, intent in enumerate(data['intents']):
        for pattern in intent['patterns']:
            try:
                # Clean the pattern text
                cleaned_pattern = clean_text(pattern)
                
                # Skip if pattern becomes empty after cleaning
                if not cleaned_pattern.strip():
                    stats['failed_patterns'] += 1
                    continue
                
                # Generate variations of the cleaned pattern
                variations = generate_pattern_variations(cleaned_pattern)
                
                # Split pattern into sentences for chunking
                chunks = sent_tokenize(cleaned_pattern)
                
                for chunk_idx, chunk in enumerate(chunks):
                    # Generate embeddings with different contexts
                    question_vector = generate_enhanced_embedding(chunk, prefix='question', model=model)
                    text_vector = generate_enhanced_embedding(chunk, prefix='text', model=model)
                    question_text_vector = generate_enhanced_embedding(chunk, prefix='combined', model=model)
                    
                    # Generate embeddings for variations and average them
                    variation_embeddings = [generate_enhanced_embedding(v, model=model) for v in variations]
                    combined_vector = np.mean(variation_embeddings, axis=0)
                    
                    # Clean the response text
                    cleaned_response = clean_text(random.choice(intent['responses']))
                    
                    # Index document
                    doc = {
                        'tag': intent['tag'],
                        'text': chunk,
                        'original_text': pattern,  # Keep original for reference
                        'question_vector_knn': question_vector.tolist(),
                        'text_vector_knn': text_vector.tolist(),
                        'question_text_vector_knn': question_text_vector.tolist(),
                        'vector_combined_knn': combined_vector.tolist(),
                        'response': cleaned_response,
                        'variations': variations
                    }
                    
                    es.index(index=index_name, id=f"{idx}_{chunk_idx}", body=doc)
                
                stats['processed_patterns'] += 1
                
                # Print sample embeddings for the first few patterns
                if idx < 5:
                    print(f"Original Pattern: {pattern}")
                    print(f"Cleaned Pattern: {cleaned_pattern}")
                    print(f"Sample Variations: {variations[:2]}")
                    print(f"Vector Samples (first 5 dimensions):")
                    print(f"Question Vector: {question_vector[:5]}")
                    print(f"Combined Vector: {combined_vector[:5]}")
                    print("-" * 40)
            
            except Exception as e:
                print(f"Error processing pattern '{pattern}': {str(e)}")
                stats['failed_patterns'] += 1
            
            progress_bar.update(1)
    
    progress_bar.close()
    
    # Print final statistics
    print("\nIndexing Statistics:")
    print(f"Total patterns processed: {stats['processed_patterns']}")
    print(f"Failed patterns: {stats['failed_patterns']}")
    print(f"Success rate: {(stats['processed_patterns']/total_patterns)*100:.2f}%")


In [15]:
def create_index(es, index_name="qa_index_2"):
    """
    Create an enhanced index with improved settings, custom analyzers, and dense_vector mappings.
    Includes support for new fields and better text analysis.
    """
    if es.indices.exists(index=index_name):
        es.indices.delete(index=index_name)
    
    mappings = {
        "settings": {
            "index": {
                "number_of_shards": 1,
                "number_of_replicas": 0,
                "refresh_interval": "1s",
                "max_ngram_diff": 50
            },
            "analysis": {
                "analyzer": {
                    "rag_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "stop",
                            "porter_stem",
                            "synonym",
                            "word_delimiter_graph",
                            "unique"
                        ]
                    },
                    "ngram_analyzer": {
                        "type": "custom",
                        "tokenizer": "standard",
                        "filter": [
                            "lowercase",
                            "stop",
                            "ngram_filter"
                        ]
                    }
                },
                "filter": {
                    "synonym": {
                        "type": "synonym",
                        "synonyms": [
                            "wellness, health, wellbeing",
                            "stress, anxiety, tension",
                            "help, assist, support",
                            "question, query, ask",
                            "feel, feeling, felt",
                            "sad, unhappy, depressed",
                            "happy, joyful, content",
                            "angry, mad, furious",
                            "tired, exhausted, fatigued",
                            "worried, concerned, troubled"
                        ]
                    },
                    "ngram_filter": {
                        "type": "ngram",
                        "min_gram": 3,
                        "max_gram": 4
                    }
                }
            }
        },
        "mappings": {
            "properties": {
                "tag": {
                    "type": "keyword"
                },
                "text": {
                    "type": "text",
                    "analyzer": "rag_analyzer",
                    "search_analyzer": "rag_analyzer",
                    "fields": {
                        "ngram": {
                            "type": "text",
                            "analyzer": "ngram_analyzer"
                        },
                        "keyword": {
                            "type": "keyword"
                        }
                    }
                },
                "original_text": {
                    "type": "text",
                    "analyzer": "rag_analyzer",
                    "fields": {
                        "keyword": {
                            "type": "keyword"
                        }
                    }
                },
                "response": {
                    "type": "text",
                    "analyzer": "rag_analyzer",
                    "fields": {
                        "keyword": {
                            "type": "keyword"
                        }
                    }
                },
                "variations": {
                    "type": "text",
                    "analyzer": "rag_analyzer"
                },
                "question_vector_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                },
                "text_vector_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                },
                "question_text_vector_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                },
                "vector_combined_knn": {
                    "type": "dense_vector",
                    "dims": 768,
                    "index": True,
                    "similarity": "cosine"
                },
                "metadata": {
                    "type": "object",
                    "enabled": True
                },
                "timestamp": {
                    "type": "date"
                }
            }
        }
    }
    
    try:
        es.indices.create(index=index_name, body=mappings)
        print(f"Index '{index_name}' created successfully with enhanced analyzers and k-NN vector fields.")
        
        # Verify index settings and mappings
        settings = es.indices.get_settings(index=index_name)
        print("\nIndex settings verified:")
        print(f"- Number of shards: {settings[index_name]['settings']['index']['number_of_shards']}")
        print(f"- Number of replicas: {settings[index_name]['settings']['index']['number_of_replicas']}")
        print("- Custom analyzers: rag_analyzer, ngram_analyzer")
        
    except Exception as e:
        print(f"Error creating index: {str(e)}")
        raise

In [16]:
create_index(es)

Index 'qa_index_2' created successfully with enhanced analyzers and k-NN vector fields.

Index settings verified:
- Number of shards: 1
- Number of replicas: 0
- Custom analyzers: rag_analyzer, ngram_analyzer


In [19]:
index_data(data, es)

Indexing Patterns:   0%|          | 0/232 [00:00<?, ?pattern/s]

Original Pattern: Hi
Cleaned Pattern: hi
Sample Variations: ['Q: hi', 'Question: hi']
Vector Samples (first 5 dimensions):
Question Vector: [ 0.03611977 -0.00055852 -0.00819155 -0.01500973 -0.02736383]
Combined Vector: [ 0.03042874 -0.01408004 -0.00665739 -0.02422638 -0.00415771]
----------------------------------------
Original Pattern: Hey
Cleaned Pattern: hey
Sample Variations: ['I want to know hey', 'Question: hey']
Vector Samples (first 5 dimensions):
Question Vector: [ 0.05952298  0.0103958  -0.02094073  0.00516158 -0.01747805]
Combined Vector: [ 0.04435221  0.01024734 -0.02064931 -0.02891829  0.00091017]
----------------------------------------
Original Pattern: Is anyone there?
Cleaned Pattern: is anyone there?
Sample Variations: ['Question: is anyone there?', 'Q: is anyone there?']
Vector Samples (first 5 dimensions):
Question Vector: [ 0.05376231 -0.06849413  0.01648517  0.02811978 -0.0335924 ]
Combined Vector: [ 0.03167063 -0.06271856  0.0120197   0.03781154 -0.01481837]
---

In [29]:
class RAGEvaluator:
    def __init__(self, es_client, index_name="qa_index_2"):
        self.es = es_client
        self.index_name = index_name
        self.evaluation_results = {}
        self.query_details = []
        
    def semantic_search(self, query: str, field: str = 'text_vector_knn', k: int = 5) -> List[Tuple[str, float]]:
        """Enhanced semantic search with better query processing."""
        # Generate query variations for better coverage
        query_variations = [
            query,
            f"question: {query}",
            f"find information about {query}",
            f"tell me about {query}",
            f"what is {query}"
        ]
        
        # Generate embeddings for all variations
        embeddings = [generate_enhanced_embedding(q, prefix='question') for q in query_variations]
        query_embedding = np.mean(embeddings, axis=0)
        
        search_query = {
            "size": k,
            "knn": {
                "field": field,
                "query_vector": query_embedding.tolist(),
                "k": k,
                "num_candidates": 100
            },
            "_source": ["text", "response", "original_text", "tag"],
            "min_score": 0.7
        }
        
        try:
            response = self.es.search(index=self.index_name, body=search_query)
            return [(hit['_source']['response'], hit['_score'], hit['_source'].get('tag', '')) 
                   for hit in response['hits']['hits']]
        except Exception as e:
            logger.error(f"Error in semantic search: {str(e)}")
            return []

    def text_search(self, query: str, k: int = 5) -> List[Tuple[str, float]]:
        """Enhanced text search with improved query analysis."""
        search_query = {
            "size": k,
            "query": {
                "bool": {
                    "should": [
                        {
                            "match": {
                                "text": {
                                    "query": query,
                                    "boost": 2.0,
                                    "fuzziness": "AUTO"
                                }
                            }
                        },
                        {
                            "match": {
                                "original_text": {
                                    "query": query,
                                    "boost": 1.5
                                }
                            }
                        },
                        {
                            "match_phrase": {
                                "text": {
                                    "query": query,
                                    "boost": 1.5,
                                    "slop": 2
                                }
                            }
                        }
                    ]
                }
            }
        }
        
        try:
            response = self.es.search(index=self.index_name, body=search_query)
            return [(hit['_source']['response'], hit['_score'], hit['_source'].get('tag', '')) 
                   for hit in response['hits']['hits']]
        except Exception as e:
            logger.error(f"Error in text search: {str(e)}")
            return []

    def combined_search(self, query: str, k: int = 5) -> List[Tuple[str, float]]:
        """Enhanced hybrid search combining text and vector approaches."""
        query_embedding = generate_enhanced_embedding(query, prefix='combined')
        
        search_query = {
            "size": k,
            "query": {
                "bool": {
                    "should": [
                        {
                            "match": {
                                "text": {
                                    "query": query,
                                    "boost": 0.3
                                }
                            }
                        }
                    ]
                }
            },
            "knn": {
                "field": "vector_combined_knn",
                "query_vector": query_embedding.tolist(),
                "k": k,
                "num_candidates": 200,
                "boost": 0.7
            }
        }
        
        try:
            response = self.es.search(index=self.index_name, body=search_query)
            return [(hit['_source']['response'], hit['_score'], hit['_source'].get('tag', '')) 
                   for hit in response['hits']['hits']]
        except Exception as e:
            logger.error(f"Error in combined search: {str(e)}")
            return []
            
    def calculate_metrics(self, results: List[Tuple[str, float]], ground_truth: str) -> Dict[str, float]:
        """Enhanced metric calculation with better response matching."""
        metrics = {
            "hit_rate": 0,
            "mrr": 0,
            "ndcg": 0,
            "precision": 0,
            "recall": 0,
            "f1": 0
        }
        
        # Debug print
        logger.info(f"\nGround Truth: {ground_truth}")
        logger.info("Top 3 Results:")
        for idx, (response, score, _) in enumerate(results[:3]):
            logger.info(f"{idx+1}. Response: {response}")
            logger.info(f"   Score: {score}")
        
        # Calculate hit rate and MRR with fuzzy matching
        for rank, (response, _, _) in enumerate(results):
            # Normalize responses for comparison
            norm_ground_truth = self._normalize_text(ground_truth)
            norm_response = self._normalize_text(response)
            
            # Calculate similarity
            similarity = self._calculate_similarity(norm_ground_truth, norm_response)
            
            if similarity > 0.8:  # Adjust threshold as needed
                metrics["hit_rate"] = 1
                metrics["mrr"] = 1 / (rank + 1)
                logger.info(f"Match found at rank {rank+1} with similarity {similarity}")
                break
        
        # Calculate NDCG with similarity scores
        relevance_scores = []
        for response, _, _ in results:
            similarity = self._calculate_similarity(
                self._normalize_text(ground_truth),
                self._normalize_text(response)
            )
            relevance_scores.append(similarity)
        
        ideal_scores = sorted(relevance_scores, reverse=True)
        try:
            metrics["ndcg"] = ndcg_score([ideal_scores], [relevance_scores])
        except Exception as e:
            logger.error(f"NDCG calculation error: {str(e)}")
            metrics["ndcg"] = 0
        
        # Calculate precision, recall, F1 with similarity threshold
        relevant_count = sum(1 for score in relevance_scores if score > 0.8)
        if relevant_count > 0:
            metrics["precision"] = relevant_count / len(results)
            metrics["recall"] = 1 if relevant_count > 0 else 0
            metrics["f1"] = 2 * (metrics["precision"] * metrics["recall"]) / (metrics["precision"] + metrics["recall"]) \
                if (metrics["precision"] + metrics["recall"]) > 0 else 0
        
        logger.info(f"Metrics for current query: {metrics}")
        return metrics
    
    def _normalize_text(self, text: str) -> str:
        """Normalize text for comparison."""
        # Convert to lowercase and remove punctuation
        text = text.lower()
        text = re.sub(r'[^\w\s]', '', text)
        # Remove extra whitespace
        text = ' '.join(text.split())
        return text
    
    def _calculate_similarity(self, text1: str, text2: str) -> float:
        """Calculate text similarity using token overlap."""
        tokens1 = set(text1.split())
        tokens2 = set(text2.split())
        
        intersection = tokens1.intersection(tokens2)
        union = tokens1.union(tokens2)
        
        if not union:
            return 0.0
        
        return len(intersection) / len(union)

    def evaluate_methods(self, test_queries: Dict[str, str], k: int = 5) -> Dict[str, Dict[str, float]]:
        """Enhanced evaluation with better debugging."""
        methods = {
            "semantic_search_question": lambda q: self.semantic_search(q, "question_vector_knn", k),
            "semantic_search_text": lambda q: self.semantic_search(q, "text_vector_knn", k),
            "semantic_search_combined": lambda q: self.semantic_search(q, "question_text_vector_knn", k),
            "text_search": lambda q: self.text_search(q, k),
            "combined_search": lambda q: self.combined_search(q, k)
        }
        
        metrics_list = ["hit_rate", "mrr", "ndcg", "precision", "recall", "f1"]
        results = {method: {metric: [] for metric in metrics_list} for method in methods}
        
        for query, ground_truth in tqdm(test_queries.items(), desc="Evaluating queries"):
            logger.info(f"\nEvaluating Query: {query}")
            logger.info(f"Ground Truth: {ground_truth}")
            
            for method_name, search_func in methods.items():
                logger.info(f"\nMethod: {method_name}")
                try:
                    search_results = search_func(query)
                    
                    if not search_results:
                        logger.warning(f"No results returned for method {method_name}")
                        continue
                    
                    metrics = self.calculate_metrics(search_results, ground_truth)
                    
                    # Store metrics
                    for metric in metrics_list:
                        results[method_name][metric].append(metrics[metric])
                    
                    # Store detailed results
                    self.query_details.append({
                        "query": query,
                        "method": method_name,
                        "ground_truth": ground_truth,
                        "top_results": search_results[:3],
                        "metrics": metrics
                    })
                    
                except Exception as e:
                    logger.error(f"Error evaluating {method_name} for query '{query}': {str(e)}")
                    logger.error(f"Traceback: {traceback.format_exc()}")
        
        # Calculate aggregated metrics
        aggregated_results = {}
        for method in results:
            method_metrics = {}
            for metric in metrics_list:
                values = results[method][metric]
                if values:
                    method_metrics[f"avg_{metric}"] = float(np.mean(values))
                    method_metrics[f"std_{metric}"] = float(np.std(values))
                else:
                    method_metrics[f"avg_{metric}"] = 0.0
                    method_metrics[f"std_{metric}"] = 0.0
            aggregated_results[method] = method_metrics
        
        self.evaluation_results = aggregated_results
        return aggregated_results

    def generate_report(self, output_file: str = None) -> None:
        """Enhanced report generation with more details."""
        if not self.evaluation_results:
            logger.warning("No evaluation results available")
            return
        
        report = ["RAG Evaluation Report"]
        report.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append("\nOverall Results:")
        
        # Create results table
        results_data = []
        for method, metrics in self.evaluation_results.items():
            result = {
                "Method": method,
                "Hit Rate": f"{metrics['avg_hit_rate']:.3f} ± {metrics['std_hit_rate']:.3f}",
                "MRR": f"{metrics['avg_mrr']:.3f} ± {metrics['std_mrr']:.3f}",
                "NDCG": f"{metrics['avg_ndcg']:.3f} ± {metrics['std_ndcg']:.3f}",
                "F1": f"{metrics['avg_f1']:.3f} ± {metrics['std_f1']:.3f}",
                "Precision": f"{metrics['avg_precision']:.3f} ± {metrics['std_precision']:.3f}",
                "Recall": f"{metrics['avg_recall']:.3f} ± {metrics['std_recall']:.3f}"
            }
            results_data.append(result)
        
        df = pd.DataFrame(results_data)
        report.append("\n" + df.to_string())
        
        # Add detailed per-query analysis
        report.append("\nDetailed Query Analysis:")
        for detail in self.query_details:
            report.append(f"\nQuery: {detail['query']}")
            report.append(f"Method: {detail['method']}")
            report.append(f"Ground Truth: {detail['ground_truth']}")
            report.append("Top 3 Results:")
            for idx, (response, score, _) in enumerate(detail['top_results']):
                report.append(f"{idx+1}. Response: {response}")
                report.append(f"   Score: {score}")
            report.append(f"Metrics: {detail['metrics']}")
            report.append("-" * 80)
        
        report_text = "\n".join(report)
        if output_file:
            with open(output_file, 'w') as f:
                f.write(report_text)
        
        print(report_text)
        return report_text

In [30]:
import traceback
# Initialize logging with more detail
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

# Initialize evaluator
evaluator = RAGEvaluator(es)

# Test queries
test_queries = {
    "How can I tell if I might have depression?": "For a diagnosis of depression, a person needs to have experienced low mood or loss of interest or pleasure in life for at least 2 weeks. Also, they will have experienced the following symptoms: feelings of sadness, hopelessness, or irritability nearly every day.",
    "What's the main difference between feeling down and depression?": "Sadness is a normal reaction to a loss, disappointment, problems, or other difficult situations. Feeling sad from time to time is just another part of being human. In these cases, feelings of sadness go away quickly and you can go about your daily life. Other ways to talk about sadness might be feeling low, feeling down, or feeling blue. A person may say they are feeling depressed, but if it goes away on its own and doesn't impact life in a big way, it probably isn't the illness of depression. Depression is a mental illness that affects your mood, the way you understand yourself, and the way you understand and relate to things around you. It can also go by different names, such as clinical depression, major depressive disorder, or major depression. Depression can come up for no reason, and it lasts for a long time. It's much more than sadness or low mood. People who experience depression may feel worthless or hopeless. They may feel unreasonable guilty. Some people may experience depression as anger or irritability. It may be hard to concentrate or make decisions. Most people lose interest in things that they used to enjoy and may isolate themselves from others. There are also physical signs of depression, such as problems with sleep, appetite and energy and unexplainable aches or pains. Some may experience difficult thoughts about death or ending their life (suicide). Depression lasts longer than two weeks, doesn't usually go away on its own, and impacts your life. It's a real illness, and it is very treatable. It's important to seek help if you're concerned about depression.",
    "What's the real difference between stress and anxiety?": "Stress and anxiety are often used interchangeably, and there is overlap between stress and anxiety. Stress is related to the same fight, flight, or freeze response as anxiety, and the physical sensations of anxiety and stress may be very similar. The cause of stress and anxiety are usually different, however. Stress focuses on mainly external pressures on us that we're finding hard to cope with. When we are stressed, we usually know what we're stressed about, and the symptoms of stress typically disappear after the stressful situation is over. Anxiety, on the other hand, isn't always as easy to figure out. Anxiety focuses on worries or fears about things that could threaten us, as well as anxiety about the anxiety itself. Stress and anxiety are both part of being human, but both can be problems if they last for a long time or have an impact on our well-being or daily life.",
    "How do I know if I need to see a mental health professional?": "If your beliefs, thoughts, feelings or behaviours have a significant impact on your ability to function in what might be considered a normal or ordinary way, it would be important to seek help.",
    "What should I know before starting mental health medication?": "The best source of information regarding medications is the physician prescribing them. He or she should be able to answer questions such as:\n1. What is the medication supposed to do?\n2. When should it begin to take effect, and how will I know when it is effective?\n3. How is the medication taken and for how long? What food, drinks, other medicines, and activities should be avoided while taking this medication?\n4. What are the side effects and what should be done if they occur?\n5. What do I do if a dose is missed?\n6. Is there any written information available about this medication?\n7. Are there other medications that might be appropriate?\n8. If so, why do you prefer the one you have chosen?\n9. How do you monitor medications and what symptoms indicate that they should be raised, lowered, or changed?\n10. All medications should be taken as directed. Most medications for mental illnesses do not work when taken irregularly, and extra doses can cause severe, sometimes dangerous side effects. Many psychiatric medications begin to have a beneficial effect only after they have been taken for several weeks.",
    "What are the early signs of mental health issues in adults?": "Symptoms of mental health disorders vary depending on the type and severity of the condition. The following is a list of general symptoms that may suggest a mental health disorder, particularly when multiple symptoms are expressed at once.\nIn adults:\nConfused thinking\nLong-lasting sadness or irritability\nExtreme highs and lows in mood\nExcessive fear, worrying, or anxiety\nSocial withdrawal\nDramatic changes in eating or sleeping habits\nStrong feelings of anger\nDelusions or hallucinations (seeing or hearing things that are not really there)\nIncreasing inability to cope with daily problems and activities\nThoughts of suicide\nDenial of obvious problems\nMany unexplained physical problems\nAbuse of drugs and/or alcohol"
}

# Run evaluation with debug output
results = evaluator.evaluate_methods(test_queries)

# Generate detailed report
evaluator.generate_report("rag_evaluation_report_detailed_4.27pm.txt")

Evaluating queries:   0%|          | 0/6 [00:00<?, ?it/s]2024-10-27 20:32:40,395 - INFO - 
Evaluating Query: How can I tell if I might have depression?
2024-10-27 20:32:40,396 - INFO - Ground Truth: For a diagnosis of depression, a person needs to have experienced low mood or loss of interest or pleasure in life for at least 2 weeks. Also, they will have experienced the following symptoms: feelings of sadness, hopelessness, or irritability nearly every day.
2024-10-27 20:32:40,397 - INFO - 
Method: semantic_search_question
2024-10-27 20:32:40,400 - INFO - Use pytorch device_name: cpu
2024-10-27 20:32:40,401 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-27 20:32:41,839 - INFO - Use pytorch device_name: cpu
2024-10-27 20:32:41,840 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-27 20:32:43,244 - INFO - Use pytorch device_name: cpu
2024-10-27 20:32:43,245 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


...

2024-10-27 20:34:58,295 - INFO - POST http://elasticsearch:9200/qa_index_2/_search [status:200 duration:0.023s]
2024-10-27 20:34:58,296 - INFO - 
Ground Truth: Symptoms of mental health disorders vary depending on the type and severity of the condition. The following is a list of general symptoms that may suggest a mental health disorder, particularly when multiple symptoms are expressed at once.
In adults:
Confused thinking
Long-lasting sadness or irritability
Extreme highs and lows in mood
Excessive fear, worrying, or anxiety
Social withdrawal
Dramatic changes in eating or sleeping habits
Strong feelings of anger
Delusions or hallucinations (seeing or hearing things that are not really there)
Increasing inability to cope with daily problems and activities
Thoughts of suicide
Denial of obvious problems
Many unexplained physical problems
Abuse of drugs and/or alcohol
2024-10-27 20:34:58,297 - INFO - Top 3 Results:
2024-10-27 20:34:58,298 - INFO - 1. Response: symptoms of mental health 

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2024-10-27 20:34:59,691 - INFO - POST http://elasticsearch:9200/qa_index_2/_search [status:200 duration:0.017s]
2024-10-27 20:34:59,699 - INFO - 
Ground Truth: Symptoms of mental health disorders vary depending on the type and severity of the condition. The following is a list of general symptoms that may suggest a mental health disorder, particularly when multiple symptoms are expressed at once.
In adults:
Confused thinking
Long-lasting sadness or irritability
Extreme highs and lows in mood
Excessive fear, worrying, or anxiety
Social withdrawal
Dramatic changes in eating or sleeping habits
Strong feelings of anger
Delusions or hallucinations (seeing or hearing things that are not really there)
Increasing inability to cope with daily problems and activities
Thoughts of suicide
Denial of obvious problems
Many unexplained physical problems
Abuse of drugs and/or alcohol
2024-10-27 20:34:59,700 - INFO - Top 3 Results:
2024-10-27 20:34:59,700 - INFO - 1. Response: symptoms of mental health 

RAG Evaluation Report
Generated on: 2024-10-27 20:34:59

Overall Results:

                     Method       Hit Rate            MRR           NDCG             F1      Precision         Recall
0  semantic_search_question  0.500 ± 0.500  0.500 ± 0.500  0.964 ± 0.048  0.167 ± 0.167  0.100 ± 0.100  0.500 ± 0.500
1      semantic_search_text  0.500 ± 0.500  0.417 ± 0.449  0.870 ± 0.120  0.167 ± 0.167  0.100 ± 0.100  0.500 ± 0.500
2  semantic_search_combined  0.500 ± 0.500  0.500 ± 0.500  0.934 ± 0.098  0.167 ± 0.167  0.100 ± 0.100  0.500 ± 0.500
3               text_search  0.667 ± 0.471  0.667 ± 0.471  0.953 ± 0.093  0.222 ± 0.157  0.133 ± 0.094  0.667 ± 0.471
4           combined_search  0.667 ± 0.471  0.667 ± 0.471  0.964 ± 0.059  0.222 ± 0.157  0.133 ± 0.094  0.667 ± 0.471

Detailed Query Analysis:

Query: How can I tell if I might have depression?
Method: semantic_search_question
Ground Truth: For a diagnosis of depression, a person needs to have experienced low mood or loss of interes

"RAG Evaluation Report\nGenerated on: 2024-10-27 20:34:59\n\nOverall Results:\n\n                     Method       Hit Rate            MRR           NDCG             F1      Precision         Recall\n0  semantic_search_question  0.500 ± 0.500  0.500 ± 0.500  0.964 ± 0.048  0.167 ± 0.167  0.100 ± 0.100  0.500 ± 0.500\n1      semantic_search_text  0.500 ± 0.500  0.417 ± 0.449  0.870 ± 0.120  0.167 ± 0.167  0.100 ± 0.100  0.500 ± 0.500\n2  semantic_search_combined  0.500 ± 0.500  0.500 ± 0.500  0.934 ± 0.098  0.167 ± 0.167  0.100 ± 0.100  0.500 ± 0.500\n3               text_search  0.667 ± 0.471  0.667 ± 0.471  0.953 ± 0.093  0.222 ± 0.157  0.133 ± 0.094  0.667 ± 0.471\n4           combined_search  0.667 ± 0.471  0.667 ± 0.471  0.964 ± 0.059  0.222 ± 0.157  0.133 ± 0.094  0.667 ± 0.471\n\nDetailed Query Analysis:\n\nQuery: How can I tell if I might have depression?\nMethod: semantic_search_question\nGround Truth: For a diagnosis of depression, a person needs to have experienced low mood o